<a href="https://colab.research.google.com/github/towardsai/ai-tutor-rag-system/blob/main/notebooks/Larger_Context_Larger_N.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Packages and Setup Variables

In [1]:
!pip install -q llama-index==0.10.37 openai==1.30.1 tiktoken==0.7.0 chromadb==0.5.0 llama-index-llms-gemini==0.1.10 llama-index-vector-stores-chroma==0.1.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import os

# Set the "OPENAI_API_KEY" in the Python environment. Will be used by OpenAI client later.
os.environ["OPENAI_API_KEY"] = "[OPENAI_API_KEY]"
os.environ["GOOGLE_API_KEY"] = "[GOOGLE_API_KEY]"

# Load Gemini Model

In [4]:
from llama_index.llms.gemini import Gemini

llm = Gemini(model="models/gemini-pro")

# Download the Vector Store

In [5]:
!wget https://github.com/AlaFalaki/tutorial_notebooks/raw/main/data/vectorstore.zip

--2024-06-07 16:54:00--  https://github.com/AlaFalaki/tutorial_notebooks/raw/main/data/vectorstore.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/vectorstore.zip [following]
--2024-06-07 16:54:01--  https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/vectorstore.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1479982 (1.4M) [application/zip]
Saving to: ‘vectorstore.zip’

vectorstore.zip     100%[===================>]   1.41M  --.-KB/s    in 0.07s   

2024-06-07 16:54:01 (21.4 MB/s) - ‘vectorstore.zip’ saved [1479982/1479982]



In [6]:
!unzip vectorstore.zip

Archive:  vectorstore.zip
   creating: mini-llama-articles/
   creating: mini-llama-articles/a361e92f-9895-41b6-ba72-4ad38e9875bd/
  inflating: mini-llama-articles/a361e92f-9895-41b6-ba72-4ad38e9875bd/data_level0.bin  
  inflating: mini-llama-articles/a361e92f-9895-41b6-ba72-4ad38e9875bd/header.bin  
 extracting: mini-llama-articles/a361e92f-9895-41b6-ba72-4ad38e9875bd/link_lists.bin  
  inflating: mini-llama-articles/a361e92f-9895-41b6-ba72-4ad38e9875bd/length.bin  
  inflating: mini-llama-articles/chroma.sqlite3  


In [7]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

# Load the vector store from the local storage.
db = chromadb.PersistentClient(path="./mini-llama-articles")
chroma_collection = db.get_or_create_collection("mini-llama-articles")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

In [8]:
from llama_index.core import VectorStoreIndex

# Create the index based on the vector store.
index = VectorStoreIndex.from_vector_store(vector_store, llm=llm)

In [9]:
for i in [2, 4, 6, 8, 10, 15, 20, 25, 30]:

  query_engine = index.as_query_engine(similarity_top_k=i)

  res = query_engine.query("How many parameters LLaMA2 model has?")

  print(f"top_{i} results:")
  print("\t", res.response)
  print("-_"*20)

top_2 results:
	 The Llama 2 model is available in four different sizes: 7 billion, 13 billion, 34 billion, and 70 billion parameters.
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
top_4 results:
	 The Llama 2 model is available in four different sizes: 7 billion, 13 billion, 34 billion, and 70 billion parameters.
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
top_6 results:
	 The Llama 2 model comes in four different sizes with varying parameter counts: 7 billion, 13 billion, 34 billion, and 70 billion parameters.
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
top_8 results:
	 The LLaMA2 model has parameter sizes of 7 billion, 13 billion, 34 billion, and 70 billion.
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
top_10 results:
	 The LLaMA2 model has four different sizes of parameters: 7 billion, 13 billion, 34 billion, and 70 billion.
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
top_15 results:
	 The LLaMA2 model has 7 billion parameters.
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
top_20 results:
	 The LLaMA2 mo

# Evaluate

In [ ]:
!wget https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/rag_eval_dataset.json

--2024-06-05 19:43:23--  https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/rag_eval_dataset.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 476714 (466K) [text/plain]
Saving to: ‘rag_eval_dataset.json’

rag_eval_dataset.js 100%[===================>] 465.54K  --.-KB/s    in 0.02s   

2024-06-05 19:43:24 (25.0 MB/s) - ‘rag_eval_dataset.json’ saved [476714/476714]



In [ ]:
# We can also load the dataset from a previously saved json file.
from llama_index.core.evaluation import EmbeddingQAFinetuneDataset

rag_eval_dataset = EmbeddingQAFinetuneDataset.from_json(
    "./rag_eval_dataset.json"
)

In [ ]:
from llama_index.core.evaluation import RelevancyEvaluator, FaithfulnessEvaluator, BatchEvalRunner
from llama_index.llms.openai import OpenAI

llm_gpt4 = OpenAI(temperature=0, model="gpt-4o")

faithfulness_evaluator = FaithfulnessEvaluator(llm=llm_gpt4)
relevancy_evaluator = RelevancyEvaluator(llm=llm_gpt4)

# Run evaluation
queries = list(rag_eval_dataset.queries.values())
batch_eval_queries = queries[:20]

runner = BatchEvalRunner(
{"faithfulness": faithfulness_evaluator, "relevancy": relevancy_evaluator},
workers=32,
)

for i in [2, 4, 6, 8, 10, 15, 20, 25, 30]:
    # Set Faithfulness and Relevancy evaluators
    query_engine = index.as_query_engine(similarity_top_k=i)

    eval_results = await runner.aevaluate_queries(
        query_engine, queries=batch_eval_queries
    )
    faithfulness_score = sum(result.passing for result in eval_results['faithfulness']) / len(eval_results['faithfulness'])
    print(f"top_{i} faithfulness_score: {faithfulness_score}")

    relevancy_score = sum(result.passing for result in eval_results['relevancy']) / len(eval_results['relevancy'])
    print(f"top_{i} relevancy_score: {relevancy_score}")

top_2 faithfulness_score: 1.0
top_2 relevancy_score: 1.0
top_4 faithfulness_score: 1.0
top_4 relevancy_score: 0.95
top_6 faithfulness_score: 1.0
top_6 relevancy_score: 0.95
top_8 faithfulness_score: 1.0
top_8 relevancy_score: 1.0
top_10 faithfulness_score: 1.0
top_10 relevancy_score: 1.0
top_15 faithfulness_score: 0.95
top_15 relevancy_score: 0.95
top_20 faithfulness_score: 1.0
top_20 relevancy_score: 0.95
top_25 faithfulness_score: 0.95
top_25 relevancy_score: 1.0
top_30 faithfulness_score: 0.95
top_30 relevancy_score: 0.95
